In [46]:
import json
from huggingface_hub import from_pretrained_keras

In [5]:
model = from_pretrained_keras("GrimSqueaker/proteinBERT")

config.json not found in HuggingFace Hub.


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

C:\Users\Meliksah\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Meliksah\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [20]:
embedding_matrix = model.weights[2].numpy()

In [22]:
embedding_matrix.shape

(26, 128)

#### Tokenization

In [23]:
# Below are taken from:
# https://github.com/nadavbra/protein_bert/blob/master/proteinbert/tokenization.py
# AA Mapping: https://www.cup.uni-muenchen.de/ch/compchem/tink/as.html
ALL_AAS = 'ACDEFGHIKLMNPQRSTUVWXY'
ADDITIONAL_TOKENS = ['<OTHER>', '<START>', '<END>', '<PAD>']

# Each sequence is added <START> and <END> tokens
ADDED_TOKENS_PER_SEQ = 2

n_aas = len(ALL_AAS)
aa_to_token_index = {aa: i for i, aa in enumerate(ALL_AAS)}
additional_token_to_index = {token: i + n_aas for i, token in enumerate(ADDITIONAL_TOKENS)}
token_to_index = {**aa_to_token_index, **additional_token_to_index}
index_to_token = {index: token for token, index in token_to_index.items()}
n_tokens = len(token_to_index)

def tokenize_seq(seq):
    other_token_index = additional_token_to_index['<OTHER>']
    return [additional_token_to_index['<START>']] + [aa_to_token_index.get(aa, other_token_index) for aa in parse_seq(seq)] + \
            [additional_token_to_index['<END>']]

In [42]:
token_to_aminoacid = {'A': 'Alanine', 'R': 'Arginine', 'N': 'Asparagine', 'D': 'Aspartic Acid',
                      'C': 'Cysteine', 'E': 'Glutamic Acid', 'Q': 'Glutamine', 'G': 'Glycine', 
                      'H': 'Histidine', 'I': 'Isoleucine', 'L': 'Leucine', 'K': 'Lysine', 'M': 'Methionine',
                      'F': 'Phenylalanine', 'P': 'Proline', 'S': 'Serine', 'T': 'Threonine', 
                      'W': 'Tryptophan', 'Y': 'Tyrosine', 'V': 'Valine', 
                      'U': 'Selenocysteine', 'O': 'Pyrrolysine',
                      'X': 'Undefined Aminoacid', '<OTHER>': '<OTHER>',
                      '<START>' : '<START>', '<END>': '<END>', '<PAD>': '<PAD>'}

In [48]:
amino_acid_to_embedding = {}
for token, index in token_to_index.items():
    amino_acid = token_to_aminoacid[token]
    print(index, token, amino_acid)
    amino_acid_to_embedding[amino_acid] = embedding_matrix[index].tolist()

0 A Alanine
1 C Cysteine
2 D Aspartic Acid
3 E Glutamic Acid
4 F Phenylalanine
5 G Glycine
6 H Histidine
7 I Isoleucine
8 K Lysine
9 L Leucine
10 M Methionine
11 N Asparagine
12 P Proline
13 Q Glutamine
14 R Arginine
15 S Serine
16 T Threonine
17 U Selenocysteine
18 V Valine
19 W Tryptophan
20 X Undefined Aminoacid
21 Y Tyrosine
22 <OTHER> <OTHER>
23 <START> <START>
24 <END> <END>
25 <PAD> <PAD>


In [49]:
with open('amino_acid_to_embedding.json', 'w') as f:
    json.dump(amino_acid_to_embedding, f)